In [1]:
%%capture
!pip install -U bitsandbytes

In [ ]:
import json
import re
from tqdm.notebook import tqdm
from transformers import pipeline
import torch
from huggingface_hub import login

In [ ]:
login('API_KEY')

In [8]:
pipe = pipeline("text-generation", model="minicreeper/Mistral-Small-24B-Instruct-2501-bnb-4bit", return_full_text=False)
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.3k [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': "I am a text-based AI model. I'm here to provide information, answer questions, and assist with tasks to the best of my ability. How can I help you today?"}]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import pandas as pd
import json

with open('/content/drive/MyDrive/HN/M2/phil_comp/odil-data-master-hn/data/terms_to_translate.json', 'r') as f:
    terms_to_translate = json.load(f)

In [11]:
values = 'nothing'

prompt_translation_1 = f"""
You are translating **key terms** from medieval-themed French data into **English**. Your task is to:

1. **Translate** each term or phrase into **accurate English**, preserving its medieval or historical context where applicable.
2. Output the result in the **same dictionary format** as the input, replacing French terms with their English translations.
3. Output the result in **strict JSON format**, with no extra text or comments.

The expected output must **only** be:

{{'nature_place_term': [...], 'character_term': [...], 'object_architecture_term': [...], 'subject_term': [...]}}

⚠️ Do NOT include any additional explanation, newlines, or text before or after the JSON.
⚠️ Keep the structure of keys exactly the same and translate **only** the list values.

# Key terms (in French): {values}
"""

In [47]:
def generate_prompt(batch):
    """
    Funzione per generare una traduzione dei termini chiave già processati in una colonna del dataset.
    """
    # Usa i valori già presenti nella colonna 'processed_values'

    prompt_translation = f"""
    You are a strict language translation assistant. I will give you a list of French terms related to medieval illustrations. Your task is to translate each French term into English, and return ONLY a valid JSON dictionary containing each French term as a key and its English translation as the value.

    ❗IMPORTANT:
    - Do NOT add any explanation, comment, or introductory text.
    - The output MUST be a valid JSON dictionary like this:
      {{
        "french_term_1": "english_translation_1",
        "french_term_2": "english_translation_2",
        ...
      }}
    - Each item in the list must be translated. Do not skip any.
    - Keep the original French spelling as keys.
    - Enclose both keys and values in double quotes. Escape special characters if necessary.
    - The output MUST be parseable by a standard JSON parser. If the format is incorrect, the response is considered invalid.

    List of French terms:{batch}
    """

    # Genera la risposta usando il pipeline
    messages = [{"role": "user", "content": prompt_translation}]

    return messages

In [57]:
prompt_list = []
batch_len = 10
for i in tqdm(range(0, len(terms_to_translate), batch_len)):
    batch = terms_to_translate[i:i+batch_len]
    prompt_list.append(generate_prompt(batch))

  0%|          | 0/198 [00:00<?, ?it/s]

In [59]:
answers = pipe(prompt_list, pad_token_id=pipe.tokenizer.eos_token_id)

In [61]:
with open('/content/drive/MyDrive/HN/M2/phil_comp/odil-data-master-hn/data/translated_terms_raw.json', 'w') as f:
    json.dump(answers, f)

In [68]:
def parse_response(raw_answer):
  text = raw_answer[0]['generated_text']
  json_string = re.findall(r'\{.*?\}', text, re.DOTALL)[-1]
  return json.loads(json_string)

In [69]:
parsed_answers = []
for i in range(len(answers)):
  parsed_answers.append(parse_response(answers[i]))

In [72]:
general_dict = {}

for batch_dict in parsed_answers:
  general_dict.update(batch_dict)

In [74]:
with open('/content/drive/MyDrive/HN/M2/phil_comp/odil-data-master-hn/data/translated_terms.json', 'w') as f:
    json.dump(general_dict, f)